# w

In [2]:
# Import:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import efficientnet_b1
from torch.utils.data import DataLoader

In [3]:
# Define hyperparameters:
num_epochs = 40
LEARNING_RATE = 0.0005

In [4]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Define CIFAR-100 data transformations for EfficientNet-B1 (using ImageNet stats)
transform = transforms.Compose([
    transforms.Resize((240, 240)),  # Resize to 240x240 for EfficientNet-B1
    transforms.ToTensor(),          # Convert to tensor (CRUCIAL!)
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet normalization (common practice)
])

In [6]:
# Load CIFAR-100 dataset
trainset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Define the EfficientNet-B1 model
class EfficientNetB1Model(nn.Module):
    def __init__(self, num_classes=100):
        super(EfficientNetB1Model, self).__init__()
        self.model = efficientnet_b1(weights="IMAGENET1K_V1")  # Load pretrained EfficientNet-B1
        self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.model(x)


In [8]:
# Initialize the model
model = EfficientNetB1Model(num_classes=100).to(device)

In [9]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [10]:
# Function to get a unique model filename (to avoid overwriting)
def get_available_filename(directory, base_name, extension=".pth"):
    """Ensure the saved model file is unique by adding a counter if needed."""
    if not os.path.exists(directory):
        os.makedirs(directory)  # Ensure directory exists
    
    file_path = os.path.join(directory, base_name + extension)
    counter = 1
    while os.path.exists(file_path):
        file_path = os.path.join(directory, f"{base_name}_{counter}{extension}")
        counter += 1
    
    return file_path

In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Evaluate on test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader):.4f}, Accuracy: {accuracy:.2f}%")


In [10]:
# Save the trained model
torch.save(model.state_dict(), "efficientnet_b1_cifar100.pth")
print("Model trained and saved successfully!")

Model trained and saved successfully!
